In [1]:
import sys, torch
print("Python:", sys.executable)
print("Torch:", torch.__version__, "CUDA wheel:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: /home/ray/miniconda3/envs/whisper-note/bin/python
Torch: 2.5.1+cu121 CUDA wheel: 12.1
CUDA available: True
GPU: NVIDIA GeForce RTX 3090


In [2]:
from huggingface_hub import whoami

print(whoami())  # confirms your saved token is being used

{'type': 'user', 'id': '6507ef88423b46492ee03543', 'name': 'rayhu', 'fullname': 'Ray Hu', 'email': 'rayhu007@gmail.com', 'emailVerified': True, 'canPay': True, 'periodEnd': 1756684799, 'isPro': False, 'avatarUrl': '/avatars/e88381835e647dbe02fbd97654539e8d.svg', 'orgs': [{'type': 'org', 'id': '636025b83605bd411c1889d9', 'name': 'Stanford', 'fullname': 'Stanford AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/604d2f473050a33ebb17ef51/Z-vDTyG_6-yZhzfXklqAK.jpeg', 'roleInOrg': 'contributor', 'isEnterprise': False}, {'type': 'org', 'id': '66087bf0abd8bd25eeffdf83', 'name': 'surfski', 'fullname': 'Surfski', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://www.gravatar.com/avatar/4fa1051e443bad5dc9df6937156a08bf?d=retro&size=100', 'roleInOrg': 'admin', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jupyter-note', 'role': 'write', 'createdAt': '2025

In [3]:
# from huggingface_hub import login
# login()

In [4]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
GPU name: NVIDIA GeForce RTX 3090


In [5]:
device = "cuda"
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

In [6]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline(
#     "text-generation", 
#     device=0, 
#     model=model_id
# )

# # torch_dtype=torch.bfloat16,      # bfloat16/float16 automatically if GPU supports it
# # device=0,               # 0 = first GPU, -1 = CPU
# # device_map="auto",       # puts model on GPU if available, or 

In [7]:

# Load model directly
from transformers import AutoTokenizer, MllamaForConditionalGeneration

# for text
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# for vision
proc = AutoProcessor.from_pretrained(model_id)


model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,   # or torch.float16
    device_map="auto",            # puts model on GPU
    low_cpu_mem_usage=True,
    attn_implementation="sdpa"    # memory-efficient attention
).eval()

model = model.to(device)   # or .to("cuda") or .to("cuda:0")
print("Model device:", model.device)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct.
403 Client Error. (Request ID: Root=1-68b28edd-48c9781143001e0a4173caa4;eddba48b-ffb9-40fd-9d16-4fdaf4c6fbeb)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-11B-Vision-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct to ask for access.

In [ ]:
prompt="User: What is 2+2?"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=300,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
text_input= "Explain the difference between supervised and unsupervised learning:"
inputs = tokenizer(text_input, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# Get logits (raw next-token scores)
with torch.no_grad():
    logits = model(**inputs).logits
    print("Logits shape:", logits.shape)   # [batch, seq_len, vocab_size]

# Convert last token logits to probabilities
probs = torch.nn.functional.softmax(logits[0, -1], dim=-1)
topk = torch.topk(probs, k=5)
print("Top-5 predicted tokens:", [tokenizer.decode([i]) for i in topk.indices.tolist()])


In [ ]:
from PIL import Image

def ask(imgs, prompt, max_new_tokens=128, temperature=0.2):
    # imgs: PIL.Image or list[PIL.Image]
    if not isinstance(imgs, list): imgs = [imgs]
    inputs = processor(text=prompt, images=imgs, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    return processor.decode(out[0], skip_special_tokens=True).strip()

In [ ]:
img = Image.open("./data/images/image.png").convert("RGB")
print(ask(img, "Describe this image in one concise sentence.", max_new_tokens=60, temperature=0.7))


In [ ]:
print(ask(img, "What color is the car? Answer in one word.", max_new_tokens=5, temperature=0.0))



In [ ]:
prompt = (
  "Extract fields as strict JSON with keys: merchant, date, total, items[]. "
  "For items, include name and price. If unknown, use null. Output JSON only."
)
print(ask(Image.open("receipt.jpg").convert("RGB"), prompt, max_new_tokens=250, temperature=0.0))


In [ ]:
print(ask(Image.open("chart.png").convert("RGB"),
          "What is the overall trend and which category peaks?", max_new_tokens=80, temperature=0.2))


In [ ]:
labels = ["cat", "dog", "fox", "other"]
prompt = "Classify the image into one of these labels: " + ", ".join(labels) + \
         ". Reply with the label only."
print(ask(Image.open("animal.jpg").convert("RGB"), prompt, max_new_tokens=5, temperature=0.0))


In [ ]:
img1 = Image.open("shoe_a.jpg").convert("RGB")
img2 = Image.open("shoe_b.jpg").convert("RGB")
print(ask([img1, img2],
          "Are these the same model? List 3 visual similarities and 3 differences.", 
          max_new_tokens=120, temperature=0.3))


In [ ]:
raise RuntimeError("⚠️ Manual only: uncomment this line to run cleanup")

import gc, torch

del model
gc.collect()
torch.cuda.empty_cache()